In [ ]:
%md
### Kardiaflow - Gold: Feedback Metrics

**Source:** `kardia_silver.silver_feedback_enriched`

**Targets:**
`gold_feedback_satisfaction`
`gold_feedback_tag_analysis`
`gold_feedback_encounter_match`

**Trigger:** Full snapshot overwrite each run.

**Purpose:** Evaluate patient feedback patterns. Measures satisfaction by specialty, provider, and source, analyzes tag usage with average scores, and reports how often feedback links back to encounters.

In [ ]:
from kflow.config import GOLD_DB, GOLD_DIR
from kflow.notebook_utils import init

init()

In [0]:
# Ensure Gold DB exists
spark.sql(f"CREATE DATABASE IF NOT EXISTS {GOLD_DB}")
spark.sql(f"USE {GOLD_DB}")

In [0]:
%sql
-- Satisfaction metrics
CREATE OR REPLACE TABLE gold_feedback_satisfaction
USING DELTA
LOCATION '${GOLD_DIR}/gold_feedback_satisfaction'
AS
SELECT
  COALESCE(provider_specialty, 'Unknown') AS provider_specialty,
  COALESCE(provider_id, 'Unknown')        AS provider_id,
  COALESCE(source, 'Unknown')             AS source,
  COUNT(*)                                AS feedback_count,
  ROUND(AVG(satisfaction_score), 2)       AS avg_score
FROM kardia_silver.silver_feedback_enriched
GROUP BY COALESCE(provider_specialty, 'Unknown'),
         COALESCE(provider_id, 'Unknown'),
         COALESCE(source, 'Unknown');

In [0]:
%sql  
-- 3. Tag usage and average satisfaction per tag  
CREATE OR REPLACE TABLE gold_feedback_tag_analysis
USING DELTA
LOCATION '${GOLD_DIR}/gold_feedback_tag_analysis'
AS
WITH exploded_tags AS (
  SELECT
    satisfaction_score,
    EXPLODE(tags) AS tag
  FROM kardia_silver.silver_feedback_enriched
)

SELECT
  tag,
  COUNT(*) AS feedback_count,
  ROUND(AVG(satisfaction_score), 2) AS avg_score
FROM exploded_tags
GROUP BY tag
ORDER BY feedback_count DESC;

In [0]:
%sql  
-- 4. Encounter–feedback match rate (has valid visit_id?)  
CREATE OR REPLACE TABLE gold_feedback_encounter_match
USING DELTA
LOCATION '${GOLD_DIR}/gold_feedback_encounter_match'
AS
WITH feedback_flag AS (
  SELECT
    CASE
      WHEN visit_id IS NOT NULL AND TRIM(visit_id) <> '' THEN 1
      ELSE 0
    END AS has_valid_visit
  FROM kardia_silver.silver_feedback_enriched
),

summary AS (
  SELECT
    COUNT(*) AS total_feedback_submissions,
    SUM(has_valid_visit) AS matched_to_encounter_count
  FROM feedback_flag
)

SELECT
  total_feedback_submissions,
  matched_to_encounter_count,
  ROUND(
    100.0 * matched_to_encounter_count / total_feedback_submissions,
    2
  ) AS match_rate_percent
FROM summary;

In [0]:
%sql  
-- 5. Preview: satisfaction metrics  
SELECT *  
FROM gold_feedback_satisfaction
ORDER BY provider_specialty, source  
LIMIT 20;

In [0]:
%sql  
-- Preview: tag analysis  
SELECT * FROM gold_feedback_tag_analysis LIMIT 10;

In [0]:
%sql  
-- Preview: encounter–feedback match rate  
SELECT * FROM gold_feedback_encounter_match LIMIT 10;